# AVALIAÇÃO 3 - PROGRAMAÇÃO EM FINANÇAS
#### Professor: Julio Russo
#### Disciplina: Programação em Finanças
#### Turma: 2025.1
#### Alunos: Lucas dos Santos Marques, Luiz Fernando M. Meirinho , Victor A. S. de Souza e Victor Flávio P. Dornelos
#### Data: 25/06/2025

## 1 - Preparando os dados

In [471]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
from tabulate import tabulate
import time

In [472]:
# Carteira brasileira
acoes_br = ['BBAS3.SA','VALE3.SA','EMBR3.SA','RANI3.SA']
# Baixar os dados
df_br = yf.download(acoes_br,auto_adjust=True,start='2014-01-01',end='2024-12-31')['Close']

# Salvando em CSV
df_br.to_csv('carteira_brasileira.csv')

[*********************100%***********************]  4 of 4 completed


In [473]:
# Carteira EUA corrigida (apenas tickers válidos)
acoes_eua = ['AAPL', 'JPM', 'AMZN', 'KO', 'BRK-B', 'XOM']

# Baixar os dados de fechamento ajustado
df_eua = yf.download(acoes_eua, auto_adjust=True, start='2014-01-01', end='2024-12-31')['Close']

# Salvar em CSV
df_eua.to_csv('carteira_eua.csv')

[*********************100%***********************]  6 of 6 completed


## PARTE 1 - AÇÕES BRASIL

### QUESTÃO 1:
Formação de portfólio com ações brasileiras (mínimo 4 ativos). Não é necessário ter os mesmo pesos iniciais.


In [474]:
# Importando os dados
df_br = pd.read_csv('carteira_brasileira.csv', index_col='Date', parse_dates=True)
pesos_br = np.array([0.35, 0.35, 0.15, 0.15])  

### QUESTÃO 2:
Analisar retornos, volatilidade(desv.pad) dos ativos e da carteira

In [475]:
# Retornos logarítmicos diários
retornos_diarios = np.log(df_br / df_br.shift(1)).dropna()

# Indicadores individuais
retorno_anual = retornos_diarios.mean() * 252
retorno_acumulado = (df_br.iloc[-1] / df_br.iloc[0]) - 1
desvio_padrao_anual = retornos_diarios.std() * np.sqrt(252)
coef_variacao = desvio_padrao_anual / retorno_anual

In [476]:
# Tabela por ativo
tabela_ativos = pd.DataFrame({
    'Retorno Anual (%)': retorno_anual * 100,
    'Retorno Acumulado (%)': retorno_acumulado * 100,
    'Desvio Padrão Anual (%)': desvio_padrao_anual * 100,
    'Coef. de Variação': coef_variacao
})

In [477]:
# Retorno do portfólio
retorno_portfolio_diario = retornos_diarios @ pesos_br
retorno_anual_ptf = retorno_portfolio_diario.mean() * 252
retorno_acumulado_ptf = np.exp(retorno_portfolio_diario.cumsum())[-1] - 1
desvio_padrao_ptf = retorno_portfolio_diario.std() * np.sqrt(252)
coef_variacao_ptf = desvio_padrao_ptf / retorno_anual_ptf

/var/folders/wk/k6089y_d7hv_rwb2m8vbm9cr0000gn/T/ipykernel_16761/989489004.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [478]:
# Adiciona à tabela
tabela_ativos.loc['Carteira'] = [
    retorno_anual_ptf * 100,
    retorno_acumulado_ptf * 100,
    desvio_padrao_ptf * 100,
    coef_variacao_ptf
]

In [479]:
# Arredondar para melhor visualização
tabela_ativos = tabela_ativos.round(2)

In [480]:
# Taxa livre de risco (ajuste conforme o país e período)
r_f = 0.01  # Exemplo: 12% a.a. no Brasil

# Sharpe dos ativos
sharpe_ativos = (retorno_anual - r_f) / desvio_padrao_anual

# Sharpe da carteira
sharpe_portfolio = (retorno_anual_ptf - r_f) / desvio_padrao_ptf

# Adiciona à tabela
tabela_ativos['Sharpe Ratio'] = sharpe_ativos
tabela_ativos.loc['Carteira', 'Sharpe Ratio'] = sharpe_portfolio

# Exibe com tabulate
from tabulate import tabulate
print("\n📊 Indicadores com Sharpe Ratio:\n")
print(tabulate(tabela_ativos.round(2), headers='keys', tablefmt='fancy_grid'))


📊 Indicadores com Sharpe Ratio:

╒══════════╤═════════════════════╤═════════════════════════╤═══════════════════════════╤═════════════════════╤════════════════╕
│          │   Retorno Anual (%) │   Retorno Acumulado (%) │   Desvio Padrão Anual (%) │   Coef. de Variação │   Sharpe Ratio │
╞══════════╪═════════════════════╪═════════════════════════╪═══════════════════════════╪═════════════════════╪════════════════╡
│ BBAS3.SA │               12.9  │                  305.52 │                     41.51 │                3.22 │           0.29 │
├──────────┼─────────────────────┼─────────────────────────┼───────────────────────────┼─────────────────────┼────────────────┤
│ EMBR3.SA │               10.39 │                  208.57 │                     43.18 │                4.16 │           0.22 │
├──────────┼─────────────────────┼─────────────────────────┼───────────────────────────┼─────────────────────┼────────────────┤
│ RANI3.SA │               10.97 │                  228.81 │          

### QUESTÃO 3:
Construção dos gráficos em plotly de retorno e gráfico PIE das participações relativas das ações no portfólio.

In [481]:
df_pie = df_br.copy()
df_br.columns = [col.replace('.SA', '') for col in df_br.columns]  # tira .SA
df_br = df_br.reset_index().melt(id_vars='Date', var_name='Ativo', value_name='Preço')

In [482]:
# Criar DataFrame
df_pie = pd.DataFrame({
    'Ativo': acoes_br,
    'Peso': pesos_br
})

In [483]:
# Cores personalizadas
cores_personalizadas = {
    'BBAS3': 'gold',         # amarelo
    'VALE3': 'red',          # vermelho
    'EMBR3': 'darkblue',     # azul escuro
    'RANI3': 'green'         # verde
}


In [484]:
# Gráfico de pizza
fig = px.pie(
    df_pie,
    names='Ativo',
    values='Peso',
    title='Participações Relativas das Ações no Portfólio',
    color='Ativo',
    color_discrete_map=cores_personalizadas,
    width=800,
    height=600
)

fig.update_traces(textinfo='percent+label', hole=0)  # hole=0 remove o buraco

fig.show()

In [485]:
# Resetar índice para usar o plotly.express
df_plot = df_br.copy()

In [486]:
# Criar gráfico
fig = px.line(
    df_plot,
    x='Date',
    y='Preço',
    color='Ativo',
    title='Evolução do Preço das Ações no Portfólio',
    color_discrete_map=cores_personalizadas
)

# Layout com barra de rolagem e hover unificado
fig.update_layout(
    hovermode='x unified',
    xaxis=dict(
        rangeslider=dict(visible=True),
        type='date'
    ),
    width=1500,
    height=600
)

fig.show()

In [487]:
df_br = pd.read_csv('carteira_brasileira.csv', index_col='Date', parse_dates=True)

In [488]:
# Normalizar os ativos
df_norm = (df_br / df_br.iloc[0]) * 100
df_norm = df_norm.reset_index().melt(id_vars='Date', var_name='Ativo', value_name='Rentabilidade Normalizada (%)')


In [489]:
# Calcular o valor normalizado da carteira
pesos = np.array([0.35, 0.35, 0.15, 0.15])
port_br = df_br @ pesos
port_br_norm = (port_br / port_br.iloc[0]) * 100
df_port = port_br_norm.reset_index().rename(columns={0: 'Rentabilidade Normalizada (%)'})
df_port['Ativo'] = 'Carteira'

In [490]:
# Unir dados
df_total = pd.concat([df_norm, df_port], ignore_index=True)


In [491]:
# Cores personalizadas
cores = {
    'BBAS3': 'gold',
    'VALE3': 'red',
    'EMBR3': 'darkblue',
    'RANI3': 'green',
    'Carteira': 'purple'
}

In [492]:
# TIRANDO .SA dos tickers
df_total['Ativo'] = df_total['Ativo'].str.replace('.SA', '')

In [493]:
fig = px.line(
    df_total,
    x='Date',
    y='Rentabilidade Normalizada (%)',
    color='Ativo',
    title='Rentabilidade dos Ativos e da Carteira',
    color_discrete_map=cores
)


fig.update_layout(
    hovermode='x unified',
    xaxis=dict(rangeslider=dict(visible=True), type='date'),
    width=1500,
    height=600
)

fig.show()


In [494]:
import plotly.graph_objects as go

# --------- Pizza ---------
pizza = go.Pie(
    labels=df_pie['Ativo'],
    values=df_pie['Peso'],
    name='Participações',
    marker_colors=[cores_personalizadas[i.replace('.SA', '')] for i in df_pie['Ativo']],
    textinfo='percent+label',
    hole=0
)

# --------- Preço ---------
linhas_preco = [
    go.Scatter(x=df_plot[df_plot['Ativo'] == ativo]['Date'],
               y=df_plot[df_plot['Ativo'] == ativo]['Preço'],
               name=ativo,
               mode='lines',
               line=dict(color=cores_personalizadas[ativo]))
    for ativo in df_plot['Ativo'].unique()
]

# --------- Rentabilidade ---------
linhas_rent = [
    go.Scatter(x=df_total[df_total['Ativo'] == ativo]['Date'],
               y=df_total[df_total['Ativo'] == ativo]['Rentabilidade Normalizada (%)'],
               name=ativo,
               mode='lines',
               line=dict(color=cores[ativo]))
    for ativo in df_total['Ativo'].unique()
]

# --------- Layout com posição ajustada ---------
layout = go.Layout(
    title='Visualização Interativa do Portfólio',
    hovermode='x unified',
    xaxis=dict(rangeslider=dict(visible=True), type='date'),
    height=600,
    width=1500,
    updatemenus=[
        dict(
            type='buttons',
            direction='right',
            showactive=True,
            x=0,
            xanchor='left',
            y=1.05,
            yanchor='top',
            pad={'r': 10, 't': 10},
            buttons=[
                dict(label='Pizza',
                     method='update',
                     args=[{'visible': [True] + [False]*len(linhas_preco) + [False]*len(linhas_rent)},
                           {'title': 'Participações Relativas das Ações no Portfólio',
                            'xaxis': {'visible': False},
                            'yaxis': {'visible': False}}]),
                dict(label='Preço',
                     method='update',
                     args=[{'visible': [False] + [True]*len(linhas_preco) + [False]*len(linhas_rent)},
                           {'title': 'Evolução do Preço das Ações no Portfólio',
                            'xaxis': {'visible': True, 'rangeslider': {'visible': True}},
                            'yaxis': {'visible': True}}]),
                dict(label='Rentabilidade',
                     method='update',
                     args=[{'visible': [False] + [False]*len(linhas_preco) + [True]*len(linhas_rent)},
                           {'title': 'Rentabilidade Normalizada dos Ativos e da Carteira',
                            'xaxis': {'visible': True, 'rangeslider': {'visible': True}},
                            'yaxis': {'visible': True}}])
            ]
        )
    ]
)

# --------- Figura final com todos os traces ---------
fig = go.Figure(data=[pizza] + linhas_preco + linhas_rent, layout=layout)

# Inicialmente só mostrar o gráfico de pizza
visibility = [True] + [False]*(len(linhas_preco) + len(linhas_rent))
for i, trace in enumerate(fig.data):
    trace.visible = visibility[i]

# Exibir
fig.show()